# DATA CLEANING 

# fns to import statements

In [40]:
# imports 
import pandas as pd
import numpy as np 
import yfinance as yf 
from yahoofinancials import YahooFinancials

def getIncomeStatement(ticker):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler income statement for the company
    """
    

    # import data
    yahoo_financials = YahooFinancials('AAPL')
    data = yahoo_financials.get_financial_stmts('quarterly', 'income')
    # Drill down in dictionary to get the data we want 
    dict_list = data['incomeStatementHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getBalanceSheet(ticker):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler balance for the company 
    """
    # import data
    yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts('quarterly', 'balance')
    # print(data)
    # Drill down in dictionary to get the data we want 
    dict_list = data['balanceSheetHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getCashSheet(ticker):
    """
        Function params: string ticker (the ticker of the company of choice)
        Returns: A data frame of the quartler balance for the company 
    """
    # import data
    yahoo_financials = YahooFinancials(ticker)
    data = yahoo_financials.get_financial_stmts('quarterly', 'cash')
    # Drill down in dictionary to get the data we want 
    dict_list = data['cashflowStatementHistoryQuarterly'][ticker]
    # Create the dataframe
    df = pd.concat([pd.DataFrame(i) for i in dict_list], axis=1)
    # Sort Columns 
    df = df.reindex(sorted(df.columns), axis=1)
    # rename the index
    df = df.rename_axis('date').reset_index()
    df = df.transpose()
    return df

def getPrices(ticker):
    """
    Inputs: ticker of a company 
    Outputs: the prices of the 4 quarterly statements for 2022 
    NOTE: price is technically for 12/30 bc there was no price available for 12/31
    """
    yahoo_financials = YahooFinancials(ticker)
    lst_dates_interst = ['2022-03-31', '2022-06-30','2022-09-30', '2022-12-30']
    lst_end_dates = ['2022-4-1', '2022-7-1', '2022-10-1', '2023-1-2']

    # get first data so that it can be concated later on 
    data = yahoo_financials.get_historical_price_data(start_date=lst_dates_interst[0], end_date=lst_end_dates[0], time_interval='daily')
    df = pd.DataFrame(data[ticker]['prices'])

    # go through and get the prices for each data and add it to the df
    for i in range(len(lst_dates_interst)-1):
        data = yahoo_financials.get_historical_price_data(start_date=lst_dates_interst[i+1], end_date=lst_end_dates[i+1], time_interval='daily')
        temp = pd.DataFrame(data[ticker]['prices'])
        df = pd.concat([df, temp], axis=0)
    # Set dates to be axis. Note price is technically for 12/30 bc there was no price available for 12/31
    df = df.set_axis(['2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31'], axis='index')

    # get rid of date columns because they're now the index
    df = df.drop(columns=['formatted_date', 'date'], axis=0)
    return df

# fns to extract metrics of interest

In [41]:
# create functions that extract metrics 

def clean_df(df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that has actually column names and gets rid of unneccsary columns
    NOTE: this function is called by each of the three function below before extracting metrics.  
    """
    cols = df.iloc[0]
    df = df.rename(columns=cols)
    df = df.drop(['date'])
    return df

def getIncomeMetrics (df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    income_metrics = [
        'dilutedNIAvailtoComStockholders',
        'grossProfit',
        'netIncome',
        'totalExpenses',
        'totalRevenue',
        'basicEPS', 
    ]
    return df.loc[:, income_metrics]

def getBalanceSheetMetrics (df):
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    balance_metrics = [
    'cashEquivalents',
    'currentAssets',
    'currentDebt',
    'netDebt',
    'totalCapitalization',
    'totalDebt',
    'totalAssets',
    'tangibleBookValue',
    'shareIssued'
    ]
    return df.loc[:, balance_metrics]

def getCashMetrics (df): 
    """
    Inputs: a 'raw' dataframe 
    Outputs: a dataframe that contains only the metrics of interest
    """
    df = clean_df(df)
    cash_metrics = [
    'freeCashFlow',
    ]
    return df.loc[:, cash_metrics]

# fns to Calculate Metrics 

In [42]:
def marketCap (price, sharesIssued):
    return price * sharesIssued

def pricePerEarningsPerShare (price, eps):
    return price/eps

def priceToBook (price, netAssets):
    return price / netAssets

In [43]:
def getCalculatedMetrics(df):
    """
    Inputs: Takes a df 
    Outputs: The same dataframe with columns added that contian the metrics. 
    """
    temp_market, temp_price_per_eps, temp_price_to_book =  [], [], []
    for i,v in df.iterrows():
        temp_market.append(marketCap(v['open'], v['shareIssued'])) 
        temp_price_per_eps.append(pricePerEarningsPerShare(v['open'], v['basicEPS']))
        temp_price_to_book.append(priceToBook(v['open'], v['netDebt']))
    df.insert(loc = len(df.columns), column='marketCap', value=temp_market)
    df.insert(loc = len(df.columns), column='priceToEarningsRatio', value=temp_price_per_eps)
    df.insert(loc = len(df.columns), column='priceToBook', value=temp_price_to_book)
    return df

# fn to produce dataframe for a company

In [44]:
def getDfForCompany(ticker):
    """
    Inputs: ticker of compnay 
    Outputs: A dataframe that contains all the metrics of interest for this project that come from financial statements and price history (ie non calculated metrics)
    """
    df = getIncomeStatement(ticker)
    inc_df = getIncomeMetrics(pd.DataFrame(df))

    df = getBalanceSheet(ticker)
    bal_df = getBalanceSheetMetrics(df)

    df = getCashSheet(ticker)
    cash_df = getCashMetrics(df)

    price_df = getPrices(ticker)

    # merge dataframes
    merge = pd.concat([inc_df, bal_df, cash_df, price_df], axis=1)

    # add company ticker to each quarter
    merge["Company"] = [ticker for i in range(4)]

    # calculate metrics 
    merge = getCalculatedMetrics(merge)
    return merge


In [45]:
df = getDfForCompany('AAPL')
df

,dilutedNIAvailtoComStockholders,grossProfit,netIncome,totalExpenses,totalRevenue,basicEPS,cashEquivalents,currentAssets,currentDebt,netDebt,...,high,low,open,close,volume,adjclose,Company,marketCap,priceToEarningsRatio,priceToBook
2022-03-31,25010000000.0,42559000000.0,25010000000.0,67299000000.0,97278000000.0,1.54,13800000000.0,118180000000.0,16658000000.0,91883000000.0,...,178.029999,174.399994,177.839996,174.610001,103049300,173.558624,AAPL,2.882354e+12,115.480517,1.935505e-09
2022-06-30,19442000000.0,35885000000.0,19442000000.0,59883000000.0,82959000000.0,1.2,14650000000.0,112292000000.0,24991000000.0,92189000000.0,...,138.369995,133.770004,137.250000,136.720001,98964500,136.096451,AAPL,2.209091e+12,114.375000,1.488789e-09
2022-09-30,20721000000.0,38095000000.0,20721000000.0,65252000000.0,90146000000.0,1.29,5100000000.0,135405000000.0,21110000000.0,96423000000.0,...,143.100006,138.000000,141.279999,138.199997,124925300,137.760773,AAPL,2.252487e+12,109.519379,1.465211e-09
2022-12-31,95171000000.0,166871000000.0,95171000000.0,81138000000.0,387537000000.0,1.89,2627000000.0,128777000000.0,11483000000.0,90575000000.0,...,129.949997,127.430000,128.410004,129.929993,77034200,129.731918,AAPL,2.034324e+12,67.941801,1.417720e-09
